In [2]:
import os

In [3]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

2025-10-28 13:06:43.152527: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-28 13:06:43.156651: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-28 13:06:43.196905: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-28 13:06:43.196938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-28 13:06:43.198338: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [4]:
import random
from pathlib import Path
import time
import argparse
import json
import submitit
import shutil

In [5]:
from OptimizedDataGenerator import OptimizedDataGenerator
from loss import custom_loss
from models import CreateModel

/home/hep/hl2822/smart-pixels-ml/smart-pixel/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [6]:
epochs = 200
batch_size = 128
val_batch_size = 128
train_file_size = 8
val_file_size = 2
n_filters = 5
pool_size = 3
learning_rate = 0.001
early_stopping_patience = 15

In [7]:
# paths
data_directory_path = "/home/hep/hl2822/smart-pixels-ml/datasets/recon3D/"
labels_directory_path = "/home/hep/hl2822/smart-pixels-ml/datasets/labels/"

In [8]:
# Create tf record datasets
# stamp = '%08x' % random.randrange(16**8)
stamp = "d7414f9d"
output_directory = Path(f"./tfrecords_{stamp}").resolve()
os.makedirs(output_directory, exist_ok=True)
tfrecords_dir_train = Path(output_directory, f"tfrecords_train_{stamp}").resolve()
tfrecords_dir_validation_3D = Path(output_directory, f"tfrecords_validation_{stamp}").resolve()

In [9]:
# Loading training and validation data generator
training_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir=f"/home/hep/hl2822/smart-pixels-ml/tfrecords_{stamp}/tfrecords_train_{stamp}/",
    shuffle=True,
    seed=13,
    quantize=True
)

validation_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir=f"/home/hep/hl2822/smart-pixels-ml/tfrecords_{stamp}/tfrecords_validation_{stamp}/",
    shuffle=True,
    seed=13,
    quantize=True
)

In [20]:
model = CreateModel((13, 21, 2), n_filters=n_filters, pool_size=pool_size)
model.compile(optimizer=Adam(learning_rate=learning_rate), loss=custom_loss)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 13, 21, 2)]       0         
                                                                 
 q_separable_conv2d_1 (QSep  (None, 11, 19, 5)         33        
 arableConv2D)                                                   
                                                                 
 q_activation_5 (QActivatio  (None, 11, 19, 5)         0         
 n)                                                              
                                                                 
 q_conv2d_1 (QConv2D)        (None, 11, 19, 5)         30        
                                                                 
 q_activation_6 (QActivatio  (None, 11, 19, 5)         0         
 n)                                                              
                                                           

In [11]:
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, Callback

checkpoint_directory = Path(f"./checkpoints_{stamp}").resolve()
checkpoint_directory.mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist

checkpoint_filepath = Path(checkpoint_directory, 'weights.{epoch:02d}-t{loss:.2f}-v{val_loss:.2f}.hdf5').resolve()

es = EarlyStopping(patience=early_stopping_patience, restore_best_weights=True)

mcp = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
)

csv_logger_directory = Path(f"./csvlogs").resolve()
csv_logger_directory.mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist
csv_logger_filepath = Path(csv_logger_directory, f'training_log_{stamp}.csv').resolve()
csv_logger = CSVLogger(csv_logger_filepath)

print(f"✓ Checkpoint directory: {checkpoint_directory}")
print(f"✓ Checkpoint files will be saved as: weights.XX-tX.XX-vX.XX.hdf5")

✓ Checkpoint directory: /home/hep/hl2822/smart-pixels-ml/checkpoints_d7414f9d
✓ Checkpoint files will be saved as: weights.XX-tX.XX-vX.XX.hdf5


In [21]:
history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    epochs=epochs,
                    callbacks=[es, mcp, csv_logger],
                    shuffle=False,
                    verbose=1)

Epoch 1/200


InvalidArgumentError: Graph execution error:

Detected at node model_1/q_separable_conv2d_1/separable_conv2d/depthwise defined at (most recent call last):
  File "/usr/lib64/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/usr/lib64/python3.9/runpy.py", line 87, in _run_code

  File "/home/hep/hl2822/smart-pixels-ml/smart-pixel/lib64/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/hep/hl2822/smart-pixels-ml/smart-pixel/lib64/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/hep/hl2822/smart-pixels-ml/smart-pixel/lib64/python3.9/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/hep/hl2822/smart-pixels-ml/smart-pixel/lib64/python3.9/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib64/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/usr/lib64/python3.9/asyncio/base_events.py", line 1905, in _run_once

  File "/us

In [13]:
# Find the latest checkpoint file
import glob

checkpoint_directory = Path(f"./checkpoints_{stamp}").resolve()
checkpoint_files = sorted(glob.glob(str(checkpoint_directory / "weights.*.hdf5")))
checkpoint_files_directory = f"/home/hep/hl2822/smart-pixels-ml/checkpoints_{stamp}/"

checkpoint_files = sorted(glob.glob(str(checkpoint_files_directory)))

if checkpoint_files:
    latest_checkpoint = checkpoint_files[-1]
    print(f"Loading weights from: {latest_checkpoint}")
    model.load_weights(latest_checkpoint)
    
    # Extract epoch number from filename to resume training
    import re
    match = re.search(r'weights\.(\d+)-', latest_checkpoint)
    if match:
        initial_epoch = int(match.group(1)) + 1
        print(f"Resuming from epoch {initial_epoch}")
    else:
        initial_epoch = 0
else:
    print("No checkpoint files found, starting from scratch")
    initial_epoch = 0

# Now train with initial_epoch parameter
history = model.fit(x=training_generator,
                    validation_data=validation_generator,
                    callbacks=[es, mcp, csv_logger],
                    epochs=epochs,
                    initial_epoch=initial_epoch,  # Add this parameter
                    shuffle=False,
                    verbose=1)

Loading weights from: /home/hep/hl2822/smart-pixels-ml/checkpoints_d7414f9d/


2025-10-28 13:16:19.358744: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open /home/hep/hl2822/smart-pixels-ml/checkpoints_d7414f9d/: FAILED_PRECONDITION: /home/hep/hl2822/smart-pixels-ml/checkpoints_d7414f9d; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


IsADirectoryError: [Errno 21] Unable to synchronously open file (file read failed: time = Tue Oct 28 13:16:19 2025
, filename = '/home/hep/hl2822/smart-pixels-ml/checkpoints_d7414f9d/', file descriptor = 73, errno = 21, error message = 'Is a directory', buf = 0x7ffef79e0af8, total read size = 8, bytes this sub-read = 8, bytes actually read = 18446744073709551615, offset = 0)